In [161]:
import pandas as pd
cereal_nodes = pd.read_csv('computer_nodes.txt', encoding='utf-16', delimiter='\t')

In [162]:
cereal_edges = pd.read_csv('computer_edges.txt', encoding='utf-16', delimiter='\t')
import networkx as nx

In [163]:
G = nx.DiGraph()

In [167]:
costs = dict(zip(list(cereal_nodes['Stage Name']), list(cereal_nodes.apply(lambda x: x.stageTime, axis=1))))

In [4]:
remove_nodes = list(cereal_edges[cereal_edges['destinationStage'].str.contains('Retail')]['sourceStage'])

In [170]:
for i, row in cereal_edges.iterrows():
    G.add_edge(row.sourceStage, row.destinationStage)

In [171]:
demands = {}
def compute_demand(node):
    if node in demands:
        return demands[node]
    demanders = G.out_edges(node)
    if len(demanders) == 0:
        d = list(cereal_nodes[cereal_nodes['Stage Name'] == node].avgDemand)[0]
    else:
        d = sum(map(lambda x: compute_demand(x[1]), demanders))
    demands[node] = d
    return d
for x in G.nodes():
    compute_demand(x)

In [7]:
import math
std_demands = {}
def compute_std_demand(node):
    if node in std_demands:
        return std_demands[node]
    demanders = G.out_edges(node)
    if len(demanders) == 0:
        d = list(cereal_nodes[cereal_nodes['Stage Name'] == node].stdDevDemand)[0]
    else:
        d = math.sqrt(sum(map(lambda x: compute_std_demand(x[1]) ** 2, demanders)))
    std_demands[node] = d
    return d
for x in G.nodes():
    compute_std_demand(x)

In [172]:
edge_widths = {}
for e in G.edges():
    edge_widths[e] = demands[e[1]]

In [173]:
nx.set_edge_attributes(G, 'value', edge_widths)

In [10]:
for x in G.nodes():
    if x in remove_nodes or 'Retail' in x:
        G.remove_node(x)

In [11]:
def process_cost(cost_str):
    return float(cost_str[1:])

In [174]:
from collections import defaultdict

edge_dictionary = defaultdict(dict)
for i, row in cereal_edges.iterrows():
    edge_dictionary[row.sourceStage][row.destinationStage] = G[row.sourceStage][row.destinationStage]['value']

In [63]:

cereal_nodes['cost'] = cereal_nodes.apply(lambda x: process_cost(x.stageCost), axis=1)
thresholds = [0, 0.5, 1, 2, 5, 10, 20, 50, 5000]
H = nx.DiGraph()
manuf = list(filter(lambda x: 'Manuf' in x, G.nodes()))
edge_values = {}
part_tables = []
manuf_tables = []
for i in range(len(thresholds) - 1):
    subtable = cereal_nodes[(cereal_nodes.cost > thresholds[i]) & (cereal_nodes.cost <= thresholds[i + 1])]
    part_tables.append(subtable[subtable.stageClassification == 'Part'])
    manuf_tables.append(subtable[subtable.stageClassification == 'Manuf'])
for a, p in enumerate(part_tables):
    for b, m in enumerate(manuf_tables):
        edge_sum = 0
        for i, part in p.iterrows():
            for j, manuf in m.iterrows():
                if G.has_edge(part['Stage Name'], manuf['Stage Name']):
                    edge_sum += G[part['Stage Name']][manuf['Stage Name']]['value']
        if edge_sum != 0:
            H.add_edge('Part_' + str(thresholds[a]), 'Manuf_' + str(thresholds[b]))
            H['Part_' + str(thresholds[a])]['Manuf_' + str(thresholds[b])]['value'] = edge_sum
    


In [50]:
costs = cereal_nodes[cereal_nodes['Stage Name'].str.contains('Manuf')].apply(lambda x: process_cost(x.stageCost), axis=1)
from scipy.stats import describe
print(describe(costs))

DescribeResult(nobs=87, minmax=(2.4199999999999999, 98.730000000000004), mean=39.535057471264366, variance=535.10312761293767, skewness=0.8366768316693697, kurtosis=-0.17277104495444995)


In [177]:
forward_nodes = dict(zip(G.nodes(), map(lambda node: list(map(lambda x: x[1], G.out_edges(node))), G.nodes())))

In [178]:
backward_nodes = dict(zip(G.nodes(), map(lambda node: list(map(lambda x: x[0], G.in_edges(node))), G.nodes())))

In [176]:
import json
with open('computer_forward_nodes.json', 'w') as fout:
    fout.write(json.dumps(dict(edge_forward_nodes), indent=2))

In [67]:
from networkx.readwrite import json_graph
data = json_graph.node_link_data(H)
with open('aircraft_grouped.json', 'w') as fout:
    print(data)
    fout.write(json.dumps(data, indent=2))

{'directed': True, 'graph': {}, 'nodes': [{'id': 'Part_0.5'}, {'id': 'Manuf_5'}, {'id': 'Part_20'}, {'id': 'Manuf_2'}, {'id': 'Part_50'}, {'id': 'Part_10'}, {'id': 'Manuf_50'}, {'id': 'Part_0'}, {'id': 'Part_1'}, {'id': 'Part_2'}, {'id': 'Manuf_10'}, {'id': 'Part_5'}, {'id': 'Manuf_20'}], 'links': [{'source': 0, 'target': 1, 'value': 275.21000000000009}, {'source': 0, 'target': 10, 'value': 3312.7300000000023}, {'source': 0, 'target': 3, 'value': 285.93999999999977}, {'source': 0, 'target': 12, 'value': 17434.560000000056}, {'source': 0, 'target': 6, 'value': 6367.2300000000041}, {'source': 2, 'target': 1, 'value': 9.4900000000000002}, {'source': 2, 'target': 10, 'value': 95.139999999999986}, {'source': 2, 'target': 3, 'value': 9.379999999999999}, {'source': 2, 'target': 12, 'value': 659.13999999999999}, {'source': 2, 'target': 6, 'value': 223.09}, {'source': 4, 'target': 12, 'value': 116.56000000000002}, {'source': 5, 'target': 1, 'value': 9.4900000000000002}, {'source': 5, 'target': 

In [66]:
import json
data = {}
data['nodes'] = G.nodes()
data['edges'] = 

{'directed': True,
 'graph': {},
 'links': [{'source': 0, 'target': 41, 'value': 8820.0},
  {'source': 1, 'target': 42, 'value': 8820.0},
  {'source': 1, 'target': 44, 'value': 8820.0},
  {'source': 2, 'target': 42, 'value': 8820.0},
  {'source': 2, 'target': 44, 'value': 8820.0},
  {'source': 3, 'target': 42, 'value': 8820.0},
  {'source': 3, 'target': 44, 'value': 8820.0},
  {'source': 4, 'target': 85, 'value': 8820.0},
  {'source': 5, 'target': 42, 'value': 8820.0},
  {'source': 5, 'target': 44, 'value': 8820.0},
  {'source': 6, 'target': 42, 'value': 8820.0},
  {'source': 6, 'target': 44, 'value': 8820.0},
  {'source': 7, 'target': 42, 'value': 8820.0},
  {'source': 7, 'target': 44, 'value': 8820.0},
  {'source': 8, 'target': 83, 'value': 8820.0},
  {'source': 9, 'target': 84, 'value': 8820.0},
  {'source': 20, 'target': 48, 'value': 8820.0},
  {'source': 21, 'target': 48, 'value': 8820.0},
  {'source': 22, 'target': 40, 'value': 8820.0},
  {'source': 22, 'target': 41, 'value': 882